In [ ]:
import os
import cv2
import datetime
import json
import torch
import copy
import shutil
import ultralytics
from glob import glob
import re
import hashlib
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
from ultralytics import YOLO

base_path = '/home/dl.net/evezzali/Gitlab/Barcode-Localization-Benchmark/dataset/'

In [ ]:
train_val_test_split = [0.6, 0.1, 0.3]

train_val_test_split = np.array(train_val_test_split)/np.sum(train_val_test_split)
th1 = train_val_test_split[0]
th2 = th1+train_val_test_split[1]

train_files = []
val_files = []
test_files = []
img_paths = glob(f'{base_path}images/*.jpg')
img_paths.extend(glob(f'{base_path}images/*.JPG'))
img_paths = list(set(img_paths))

for img_path in img_paths:
    file_name = os.path.basename(img_path)
    hash_code = int(hashlib.sha256(file_name.encode('utf-8')).hexdigest(), 16) % 10 ** 4
    if hash_code < th1 * (10 ** 4):
        train_files.append(img_path)
    elif hash_code < th2 * (10 ** 4):
        val_files.append(img_path) 
    else:
        test_files.append(img_path)
print(len(train_files), len(val_files), len(test_files))

In [ ]:
glob(f'{base_path}/*.txt')

In [ ]:
#Generate txt files
string = ""

for img_path in train_files:
    file_name = os.path.basename(img_path)
    string = f'{string}{base_path}images/{file_name}' + '\n'

with open(f'{base_path}train.txt', 'w') as f:
    f.write(string)

string = ""

for img_path in test_files:
    file_name = os.path.basename(img_path)
    string = f'{string}{base_path}images/{file_name}' + '\n'

with open(f'{base_path}test.txt', 'w') as f:
    f.write(string)

string = ""

for img_path in val_files:
    file_name = os.path.basename(img_path)
    string = f'{string}{base_path}images/{file_name}' + '\n'

with open(f'{base_path}val.txt', 'w') as f:
    f.write(string)



In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda:1')

In [ ]:
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)
model.to(device);

In [ ]:
model.train(data=f'{base_path}data.yaml', epochs=200, imgsz=480, patience=10)

In [ ]:
import shutil
model.export(format='onnx')
shutil.move('/home/dl.net/evezzali/runs/detect/train35/weights/best.onnx', './Saved Models/yolov8m_480_2cl.onnx')

In [ ]:
model = YOLO('./Saved Models/yolov8s_trained.onnx', task='detect')

In [ ]:
file_path = f'{base_path}/yolo_dataset/images/Special_0099.jpg'
result = model(file_path)

In [ ]:
%matplotlib inline
img = cv2.imread(file_path)
H, W, _ = img.shape
for bbox in result[0].boxes.data.cpu().numpy():
    x0, y0, x1, y1, conf, pred = bbox
    if int(np.round(pred)) == 0:
        cv2.rectangle(img, pt1=(int(x0), int(y0)), pt2=(int(x1), int(y1)), color=(0,0,255), thickness=int((H+W)/500))
    else:
        cv2.rectangle(img, pt1=(int(x0), int(y0)), pt2=(int(x1), int(y1)), color=(255,0,0), thickness=int((H+W)/500))

plt.imshow(img[:,:,::-1])
plt.show()


In [ ]:
device = torch.device('cuda:1')
model = YOLO("./yolo.yaml");
model.to(device);

In [ ]:
model.train(data="coco.yaml", epochs=200, imgsz=640, patience=10)